In [138]:
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu/'

import pandas as pd
import datasets
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from datasets import Dataset, DatasetDict
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support
import wandb
import os
import evaluate
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoModelForSeq2SeqLM
from sklearn.dummy import DummyClassifier
from collections import Counter
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PrefixTuningConfig, PromptTuningConfig, TaskType, PeftType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
from peft import PeftModel, PeftConfig

import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # replace the 0 with other gpu ids
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [139]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
#seed = 1234
#seed = 20230501
#seed = 120398412
seed = 987654321
#seed = 42
    
seed_everything(seed)

In [140]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = "cuda:0"
model_name_or_path = "google/flan-t5-xl"
tokenizer_name_or_path = "google/flan-t5-xl"

text_column = "text"
label_column = "answer"
max_length = 192
lr = 1e-2
num_epochs = 20
batch_size = 32

In [141]:
datadir = '/shared/2/projects/contextual-appropriateness/data/final-annotated-data-cleaned/'
train_df = pd.read_csv(datadir + 'train.csv')
dev_df = pd.read_csv(datadir + 'dev.csv')
test_df = pd.read_csv(datadir + 'test.csv')

In [142]:
len(set(test_df.quote))

96

In [143]:
len(train_df)

9107

In [144]:
train_df.head()

,id,relationship,is_appropriate,label,quote,origin
0,594,ex_dating,Yes,0,She thought it was funny... but the animals sh...,Adjudication
1,594,colleague,No,1,She thought it was funny... but the animals sh...,Adjudication
2,594,engaged,Yes,0,She thought it was funny... but the animals sh...,Adjudication
3,594,best_friend,Yes,0,She thought it was funny... but the animals sh...,Adjudication
4,594,neighbor,No,1,She thought it was funny... but the animals sh...,Adjudication


In [145]:
#train_df = train_df.tail(4000)

In [146]:
relationship_to_verbalization = {
    "friend": "a person to their friend",
    "parent":"a parent to their child",
    "child":"a child to their parent",
    "sibling":"a person to their sibling",
    "best friend":"a person to their best friend",
    "best_friend":"a person to their best friend",
    "neighbor":"a person to their neighbor",
    "coworker":"a person to their coworker",
    "boss":"a boss to their employee",
    "direct report (employee)":"a person to their boss",
    "direct_report":"a person to their boss",    
    "student":"a student to their teacher",
    "teacher":"a teacher to their student",
    "cousins":"a person to their cousin",
    "granparent":"a person to their grandchild",
    "grandparent":"a person to their grandchild",
    "grandchild":"a person to their grandparent",    
    "uncle/aunt":"an uncle/aunt to their niece or nephew",
    "uncle_aunt":"an uncle or aunt to their niece or nephew",    
    "neice_nephew":"a niece or nephew to their uncle or aunt",        
    "employee_in_large_company":"an employee in large company to another",
    "married":"a person to their spouse",
    "dating":"a person to the person they are dating",
    "engaged":"a person to their fiancee",
    "friends_with_benefits":"a person to their friend with benefits",
    "divorcee":"a person to their ex-spouse",
    "ex-girlfriend/ex-boyfriend":"a person to their ex-girlfriend or ex-boyfriend",
    "ex_dating":"a person to their ex-girlfriend or ex-boyfriend",    
    "step-sibling":"a person to their step-sibling",
    "step_sibling":"a person to their step-sibling",    
    "fan":"a fan to their hero",
    "hero":"a hero to their fan",
    "enemy":"a person to their enemy",
    "rival":"a person to their rival",
    "competitor":"a person to their competitor",
    "complete_stranger":"a person to a complete stranger",
    "acquaintance":"a person to an acquaintance",
    "person_with_authority":"a person with authority to a subordinate",
    "law_enforcement":"a member of law enforcement to a community member",
    "classmate":"a person to their classmate",
    "sports_teammate":"a person to their sports teammate",
    "club_member":"a person to a member of their club",
    "adopted_child":"an adopted child to their parent",
    "adopted child":"an adopted child to their parent",
    "domestic_partner":"a person to their domestic partner",
    "person_having_an_affair":"a person having an affair to their partner",
    "mentor":"a mentor to their mentee",
    "mentee":"a mentee to their mentor",
    "landlord":"a landlord to their tenant",
    "colleague":"a person to their colleague",
    "childhood_friend":"a person to their childhood friend",
    "old_friend":"a person to an old friend",
    "client":"a client to their lawyer",
    "lawyer":"a lawyer to their client",
    "patient":"a patient to their doctor",
    "doctor":"a doctor to their patient",   
}

In [147]:
def generate_text(row):
    rel = row['relationship']
    if rel in relationship_to_verbalization:
        desc = relationship_to_verbalization[rel]
    else:
        desc = relationship_to_verbalization[rel.replace(' ', '_')]
    text = 'Rate whether it is inappropriate for this message to be said in the following social setting?\nsetting: from %s\nmessage: %s\nanswer (yes or no):' % (desc, row['quote'])
    return text

In [148]:
#set(train_df.relationship)

In [149]:
train_df['text'] = train_df.apply(generate_text, axis=1)
dev_df['text'] = dev_df.apply(generate_text, axis=1)
test_df['text'] = test_df.apply(generate_text, axis=1)
train_df.head()

,id,relationship,is_appropriate,label,quote,origin,text
0,594,ex_dating,Yes,0,She thought it was funny... but the animals sh...,Adjudication,Rate whether it is inappropriate for this mess...
1,594,colleague,No,1,She thought it was funny... but the animals sh...,Adjudication,Rate whether it is inappropriate for this mess...
2,594,engaged,Yes,0,She thought it was funny... but the animals sh...,Adjudication,Rate whether it is inappropriate for this mess...
3,594,best_friend,Yes,0,She thought it was funny... but the animals sh...,Adjudication,Rate whether it is inappropriate for this mess...
4,594,neighbor,No,1,She thought it was funny... but the animals sh...,Adjudication,Rate whether it is inappropriate for this mess...


In [150]:
train_df['answer'] = train_df.label.apply(lambda x: 'yes' if x == 1 else 'no')
dev_df['answer'] = dev_df.label.apply(lambda x: 'yes' if x == 1 else 'no')
test_df['answer'] = test_df.label.apply(lambda x: 'yes' if x == 1 else 'no')

In [151]:
print(test_df.text.iloc[0])

Rate whether it is inappropriate for this message to be said in the following social setting?
setting: from a person to their ex-girlfriend or ex-boyfriend
message: He needs to get high school diploma too
answer (yes or no):


In [152]:
tds = Dataset.from_pandas(train_df)
vds = Dataset.from_pandas(dev_df)
test_ds = Dataset.from_pandas(test_df)

ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds
ds['test'] = test_ds

dataset = ds
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'relationship', 'is_appropriate', 'label', 'quote', 'origin', 'text', 'answer'],
        num_rows: 9107
    })
    validation: Dataset({
        features: ['id', 'relationship', 'is_appropriate', 'label', 'quote', 'origin', 'text', 'answer'],
        num_rows: 1100
    })
    test: Dataset({
        features: ['id', 'relationship', 'is_appropriate', 'label', 'quote', 'origin', 'text', 'answer'],
        num_rows: 2029
    })
})

In [153]:
torch.cuda.empty_cache()

In [154]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=2, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs

In [155]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/9107 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/1100 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/2029 [00:00<?, ? examples/s]

In [156]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]
eval_df = dev_df
test_dataset = processed_datasets["test"]


train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, 
    batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, 
                             batch_size=batch_size, pin_memory=True)

In [157]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, 
                                 inference_mode=False, num_virtual_tokens=20)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 1966080 || all params: 2851723264 || trainable%: 0.06894357614638445


In [158]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [159]:
model = model.to(device)
outdir = '/shared/2/projects/contextual-appropriateness/models/peft/%s/seed-%d/' % (model_name_or_path, seed)

best_f1 = 0
best_model = 0

for epoch in trange(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), 
                                   skip_special_tokens=True)
        )

    eval_bin_preds = [1 if p == 'yes' else 0 for p in eval_preds]
    p,r,f1,s = precision_recall_fscore_support(eval_df['label'], eval_bin_preds, average='binary')


    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")
    print('Precision: %.3f, Recall: %.3f, F1: %.3f' % (p,r,f1))
            
    if best_f1 < f1:
        best_f1 = f1
        print('saving best model (currently: epoch %d)' % epoch)
        model.save_pretrained(outdir + 'best/')
    model.save_pretrained(outdir + 'epoch-%03d/' % epoch)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/285 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

epoch=0: train_ppl=tensor(1.3821, device='cuda:0') train_epoch_loss=tensor(0.3236, device='cuda:0') eval_ppl=tensor(1.3545, device='cuda:0') eval_epoch_loss=tensor(0.3034, device='cuda:0')
Precision: 0.637, Recall: 0.539, F1: 0.584
saving best model (currently: epoch 0)


  0%|          | 0/285 [00:00<?, ?it/s]

...

  0%|          | 0/35 [00:00<?, ?it/s]

epoch=18: train_ppl=tensor(1.1520, device='cuda:0') train_epoch_loss=tensor(0.1415, device='cuda:0') eval_ppl=tensor(1.3689, device='cuda:0') eval_epoch_loss=tensor(0.3140, device='cuda:0')
Precision: 0.752, Recall: 0.656, F1: 0.701


  0%|          | 0/285 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

epoch=19: train_ppl=tensor(1.1515, device='cuda:0') train_epoch_loss=tensor(0.1411, device='cuda:0') eval_ppl=tensor(1.3739, device='cuda:0') eval_epoch_loss=tensor(0.3176, device='cuda:0')
Precision: 0.748, Recall: 0.654, F1: 0.698


# Load in best model

In [160]:
best_model_dir = '/shared/2/projects/contextual-appropriateness/models/peft/%s/seed-%d/best' \
    % (model_name_or_path, seed)

config = PeftConfig.from_pretrained(best_model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, best_model_dir)
model.to(device)

test_dataloader = DataLoader(test_dataset, collate_fn=default_data_collator, 
                             batch_size=batch_size, pin_memory=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [161]:
model.eval()
eval_preds = []
for step, batch in enumerate(tqdm(test_dataloader)):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    eval_preds.extend(
        tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), 
                               skip_special_tokens=True)
    )

eval_bin_preds = [1 if p == 'yes' else 0 for p in eval_preds]

  0%|          | 0/64 [00:00<?, ?it/s]

In [162]:
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["test"]["answer"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{dataset['test']['answer'][:10]=}")

accuracy=72.10448496796451 % on the evaluation dataset
eval_preds[:10]=['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no']
dataset['test']['answer'][:10]=['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no']


In [163]:
test_preds = [1 if p == 'yes' else 0 for p in eval_preds]
test_df2 = test_df.copy()
test_df2['predicted'] = test_preds

In [164]:
def score(sdf):
    #f1 = f1_score(sdf.actual, sdf.predicted, average='binary')
    #cat = list(set(sdf.category))[0]
    p, r, f1, sup = precision_recall_fscore_support(sdf.label, sdf.predicted, average='binary')
    return pd.Series({'precision': p, 'recall': r, 'f1': f1, 'training examples': len(sdf)})

res = test_df2.groupby('relationship').apply(score).sort_values(by='f1', ascending=False).reset_index()
res.head()

,relationship,precision,recall,f1,training examples
0,hero,1.000000,1.000000,1.000000,5.0
1,client,0.833333,1.000000,0.909091,8.0
2,teacher,0.857143,0.923077,0.888889,37.0
3,patient,0.800000,1.000000,0.888889,6.0
4,boss,0.811594,0.965517,0.881890,74.0


In [165]:
res.tail(10)

,relationship,precision,recall,f1,training examples
39,friends_with_benefits,0.333333,0.181818,0.235294,54.0
40,engaged,0.333333,0.181818,0.235294,58.0
41,person_having_an_affair,0.400000,0.166667,0.235294,50.0
42,cousins,0.142857,0.142857,0.142857,56.0
43,old_friend,0.181818,0.095238,0.125000,65.0
44,childhood_friend,0.125000,0.055556,0.076923,64.0
45,sibling,0.000000,0.000000,0.000000,61.0
46,step_sibling,0.000000,0.000000,0.000000,56.0
47,best_friend,0.000000,0.000000,0.000000,80.0
48,friend,0.000000,0.000000,0.000000,72.0


In [166]:
res[res.columns[1:]].corr()

,precision,recall,f1,training examples
precision,1.000000,0.874682,0.954489,-0.546523
recall,0.874682,1.000000,0.977124,-0.568010
f1,0.954489,0.977124,1.000000,-0.578116
training examples,-0.546523,-0.568010,-0.578116,1.000000


# Note that this is *one seed's* scores; the paper reports the mean across all 5 seeds

In [167]:
precision_recall_fscore_support(test_df2.label, test_df2.predicted, average='binary')

(0.6751662971175166, 0.6904761904761905, 0.6827354260089686, None)

## 